In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy
%matplotlib inline
import pylab
from numpy import array, pi
from astropy.io import fits

In [5]:
from astropy.table import Table
t = Table.read('LOFAR2.fits', format='fits')

In [6]:
print(t)

     Source_Name            Total_flux     ...        z_best      
---------------------- ------------------- ... -------------------
ILTJ104327.95+521032.6  0.5841184590020289 ...              0.6157
ILTJ104330.98+515535.8   2.233776166523432 ...              1.0181
ILTJ104332.52+515259.6  0.9024277866188777 ...              0.5481
ILTJ104333.37+520010.8  0.5899020578887203 ...  0.4662609100341797
ILTJ104341.17+514713.0  0.6132727376440176 ...               0.485
ILTJ104341.64+522025.7   42.75226347356401 ...              0.7929
ILTJ104342.09+521137.2  1.1169500581605294 ... 0.44418030977249146
ILTJ104342.94+514316.7  0.5127128820681194 ...              0.2586
ILTJ104345.35+523239.2  2.9488830743857526 ...              0.2335
ILTJ104346.26+521856.8 0.43031970452588736 ...              1.0163
                   ...                 ... ...                 ...
ILTJ152321.22+521306.1  1.6301366464419211 ...   2.094599962234497
ILTJ152324.56+532102.0    3.66562697410943 ...              0.

In [7]:
hdul = fits.open('LOFAR2.fits')
data = hdul[1].data
flux_tot = data.field('Total_flux') # Obtain the total flux column 
z = data.field('z_best') # Obtain redshift column

In [49]:
#Import package in order to get the function for Luminosity

from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)

In [50]:
# Luminosity Distance
Lum_d = cosmo.luminosity_distance(flux_tot)*3.08567758e24 

In [51]:
# Luminosity based on redshift and distance
def power(d,f,z):
    P = (4*pi*(d)**2*f*1e-20/(1+z))
    return P
Pw = power(Lum_d,flux_tot,z)

In [54]:
Pw_adj = filter(lambda x: x >= 4e43, Pw)

AttributeError: 'filter' object has no attribute 'shape'